In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

pd.set_option('display.max_columns', None)


In [3]:
# Charger tous les fichiers DVF du dossier "Data/" (accept: .csv et .txt)
import glob
from pathlib import Path

# Les fichiers DVF fournis ici sont en .txt séparés par '|' — on supporte aussi .csv si présent.
patterns = ["Data/*.csv", "Data/*.txt"]
files = []
for p in patterns:
    files.extend(glob.glob(p))

# Tri et suppression des doublons (utile si plusieurs extensions ou erreurs de copie)
files = sorted(set(files))

if not files:
    print("⚠️ Aucun fichier trouvé dans Data/. Vérifie ton chemin ou l’extension (.csv/.txt).")
else:
    df_list = []
    for f in files:
        try:
            # Les fichiers DVF sont en général séparés par '|' — on précise un fallback d'encodage si nécessaire
            df_list.append(pd.read_csv(f, sep='|', low_memory=False, encoding='utf-8'))
        except Exception as e:
            print(f"⚠️ Échec lecture {f}: {e}. Réessai avec 'latin-1'...")
            df_list.append(pd.read_csv(f, sep='|', low_memory=False, encoding='latin-1'))
    df = pd.concat(df_list, ignore_index=True)
    print(f"✅ {len(df):,} lignes chargées depuis {len(files)} fichier(s).")
    print("Aperçu des données :")
    try:
        display(df.head())
    except NameError:
        print(df.head())

✅ 20,133,668 lignes chargées depuis 5 fichier(s).
Aperçu des données :


,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,"8000,00",NaN,NaN,NaN,B063,FORTUNAT,1250.0,CEYZERIAT,01,72,NaN,AK,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,"2175,00",NaN,NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,LAIZ,01,203,NaN,B,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,"2175,00",NaN,NaN,NaN,B006,BOIS DU CHAMP RION,1290.0,LAIZ,01,203,NaN,B,173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,"2175,00",NaN,NaN,NaN,B025,EN COROBERT,1290.0,LAIZ,01,203,NaN,B,477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,"2175,00",NaN,NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,LAIZ,01,203,NaN,C,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


In [4]:
# 🧹 Étape 2 — Sélection + nettoyage léger

# Normalisation des noms de colonnes
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
)

# ✅ Colonnes à garder pour l’analyse
colonnes_utiles = [
    'date_mutation',
    'nature_mutation',
    'valeur_fonciere',
    'code_postal',
    'commune',
    'type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales'
]

# Vérifier que toutes existent
colonnes_presentes = [c for c in colonnes_utiles if c in df.columns]
df = df[colonnes_presentes].copy()

print(f"📦 {len(colonnes_presentes)} colonnes gardées : {colonnes_presentes}")

# 🎯 Filtrer uniquement les ventes
df = df[df['nature_mutation'].str.contains('Vente', case=False, na=False)]

# 💰 Nettoyage des montants
df['valeur_fonciere'] = (
    df['valeur_fonciere']
    .astype(str)
    .str.replace(',', '.')
    .str.replace(' ', '')
    .astype(float)
)

# 📏 Nettoyage des surfaces
df['surface_reelle_bati'] = (
    df['surface_reelle_bati']
    .fillna(0)
    .astype(str)
    .str.replace(',', '.')
    .replace('', '0')
    .astype(float)
)

# 🚫 Suppression des lignes sans code postal
df = df.dropna(subset=['code_postal'])

print(f"✅ Après sélection et nettoyage : {len(df):,} lignes restantes.")
df.head(5)


📦 8 colonnes gardées : ['date_mutation', 'nature_mutation', 'valeur_fonciere', 'code_postal', 'commune', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales']
✅ Après sélection et nettoyage : 19,689,031 lignes restantes.


,date_mutation,nature_mutation,valeur_fonciere,code_postal,commune,type_local,surface_reelle_bati,nombre_pieces_principales
0,07/01/2020,Vente,8000.0,1250.0,CEYZERIAT,NaN,0.0,NaN
1,02/01/2020,Vente,2175.0,1290.0,LAIZ,NaN,0.0,NaN
2,02/01/2020,Vente,2175.0,1290.0,LAIZ,NaN,0.0,NaN
3,02/01/2020,Vente,2175.0,1290.0,LAIZ,NaN,0.0,NaN
4,02/01/2020,Vente,2175.0,1290.0,LAIZ,NaN,0.0,NaN


In [5]:
# 💰 Calcul du prix au m²
df['prix_m2'] = np.where(
    df['surface_reelle_bati'] > 0,
    df['valeur_fonciere'] / df['surface_reelle_bati'],
    np.nan
)

# 🧹 Suppression des valeurs aberrantes (trop basses ou trop hautes)
df = df[(df['prix_m2'] > 500) & (df['prix_m2'] < 20000)]


In [6]:
# 🎯 Critères de base pour logements étudiants
df_student = df.copy()

# Type d’appartement : studio / petit appart
df_student = df_student[df_student['type_local'].str.contains('Appartement', case=False, na=False)]

# Petites surfaces typiques (10–35 m²)
df_student = df_student[(df_student['surface_reelle_bati'] >= 10) & (df_student['surface_reelle_bati'] <= 35)]

# 1 pièce principale max
df_student = df_student[df_student['nombre_pieces_principales'] <= 1]

# Retrait des prix hors norme pour les petites surfaces
df_student = df_student[(df_student['prix_m2'] < df_student['prix_m2'].quantile(0.99))]

print(f"🏡 Nombre de logements étudiants potentiels : {len(df_student):,}")
df_student.head(10)


🏡 Nombre de logements étudiants potentiels : 390,171


,date_mutation,nature_mutation,valeur_fonciere,code_postal,commune,type_local,surface_reelle_bati,nombre_pieces_principales,prix_m2
14,06/01/2020,Vente,54800.0,1000.0,BOURG-EN-BRESSE,Appartement,32.0,1.0,1712.500000
164,20/01/2020,Vente,42000.0,1000.0,BOURG-EN-BRESSE,Appartement,33.0,1.0,1272.727273
407,23/01/2020,Vente,50000.0,1000.0,BOURG-EN-BRESSE,Appartement,22.0,1.0,2272.727273
861,07/02/2020,Vente,55000.0,1440.0,VIRIAT,Appartement,29.0,1.0,1896.551724
1115,20/02/2020,Vente,259000.0,1000.0,BOURG-EN-BRESSE,Appartement,25.0,1.0,10360.000000
1400,14/02/2020,Vente,178300.0,1000.0,BOURG-EN-BRESSE,Appartement,20.0,1.0,8915.000000
1421,12/03/2020,Vente,49500.0,1000.0,BOURG-EN-BRESSE,Appartement,20.0,1.0,2475.000000
1642,13/03/2020,Vente,52000.0,1000.0,BOURG-EN-BRESSE,Appartement,35.0,1.0,1485.714286
1661,12/03/2020,Vente,60000.0,1000.0,BOURG-EN-BRESSE,Appartement,35.0,1.0,1714.285714
1744,06/03/2020,Vente,61500.0,1000.0,BOURG-EN-BRESSE,Appartement,23.0,1.0,2673.913043


In [7]:
# 🏙️ Étape 3 — Filtrage géographique sur zones étudiantes
codes_postaux_cibles = [
    # Île-de-France (Paris + proche banlieue)
    75000, 75001, 75002, 75003, 75004, 75005, 75006, 75007, 75008, 75009,
    75010, 75011, 75012, 75013, 75014, 75015, 75016, 75017, 75018, 75019, 75020,
    92100, 92200, 92300, 92400, 92500, 92600, 92700, 92800, 93100, 93200, 93300, 93400, 93500, 93600, 94000, 94100, 94200, 94300, 94400, 94500, 94600, 94700, 94800,

    # Lyon
    69000, 69100, 69200, 69300, 69400, 69500, 69600,

    # Marseille & Aix-en-Provence
    13000, 13100, 13200, 13300, 13400, 13500, 13600, 13700, 13800, 13900,

    # Toulouse
    31000, 31100, 31200, 31300, 31400, 31500,

    # Lille
    59000, 59100, 59200, 59300, 59400, 59500, 59600, 59700, 59800,

    # Bordeaux
    33000, 33100, 33200, 33300, 33400, 33500, 33600, 33700, 33800, 33900,

    # Montpellier
    34000, 34100, 34200, 34300, 34400, 34500, 34600, 34700, 34800, 34900,

    # Nantes
    44000, 44100, 44200, 44300, 44400, 44500, 44600, 44700, 44800, 44900,

    # Strasbourg
    67000, 67100, 67200, 67300, 67400, 67500, 67600, 67700, 67800, 67900,

    # Rennes
    35000, 35100, 35200, 35300, 35400, 35500, 35600, 35700, 35800, 35900,

    # Grenoble
    38000, 38100, 38200, 38300, 38400, 38500, 38600, 38700, 38800, 38900,   
    
]

# ⚙️ Conversion du code postal
df['code_postal'] = df['code_postal'].astype(str).str[:5]
df['code_postal'] = df['code_postal'].astype(float)

# 🎯 Filtrage sur les zones étudiantes
df_student_typique = df[df['code_postal'].isin(codes_postaux_cibles)].copy()

print(f"🏙️ Biens situés dans les zones étudiantes : {len(df_student_typique):,}")
df_student_typique[['commune', 'code_postal', 'valeur_fonciere', 'surface_reelle_bati']].head(10)


🏙️ Biens situés dans les zones étudiantes : 693,189


,commune,code_postal,valeur_fonciere,surface_reelle_bati
336087,AUBAGNE,13400.0,90000.0,47.0
336096,AUBAGNE,13400.0,157500.0,64.0
336106,AUBAGNE,13400.0,140000.0,68.0
336109,AUBAGNE,13400.0,85000.0,28.0
336116,AUBAGNE,13400.0,716300.0,116.0
336156,LA CIOTAT,13600.0,323000.0,60.0
336170,LA CIOTAT,13600.0,300000.0,67.0
336185,AUBAGNE,13400.0,106000.0,34.0
336203,LA CIOTAT,13600.0,180000.0,50.0
336205,LA CIOTAT,13600.0,204000.0,42.0


In [8]:
print(f"Biens zones étudiantes : {len(df_student):,}")
print(f"Logements étudiants typiques : {len(df_student_typique):,}")

print(f"➡️ Ratio logements adaptés / zones étudiantes = {(len(df_student_typique) / len(df_student) * 100):.2f}%")


Biens zones étudiantes : 390,171
Logements étudiants typiques : 693,189
➡️ Ratio logements adaptés / zones étudiantes = 177.66%


In [9]:
prix_moyen_ville = (
    df_student.groupby('commune')['prix_m2']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

prix_moyen_ville.head(15)



,commune,prix_m2
0,CRUET,18472.222222
1,CAPESTERRE DE MARIE GALANTE,18392.857143
2,CLERES,18285.714286
3,LES MARTRES D ARTIERE,18224.000000
4,COUX,18185.000000
5,EYRAGUES,18166.666667
6,SAINT-QUAY-PERROS,18166.666667
7,BUXIERES SUR ARCE,18129.166667
8,OUZOUS,18107.500000
9,COUX ET BIGAROQUE-MOUZENS,18055.555556


In [10]:
# 🧩 Fusion des deux datasets : on garde uniquement les logements étudiants situés dans les zones étudiantes
df_student_final = pd.merge(
    df_student_typique,
    df_student,
    how='inner',
    on=['date_mutation', 'commune', 'code_postal', 'valeur_fonciere', 'surface_reelle_bati'],
    suffixes=('_typique', '_zone')
)

print(f"✅ Fusion réalisée : {len(df_student_final):,} lignes finales")
df_student_final.head()


✅ Fusion réalisée : 107,172 lignes finales


,date_mutation,nature_mutation_typique,valeur_fonciere,code_postal,commune,type_local_typique,surface_reelle_bati,nombre_pieces_principales_typique,prix_m2_typique,nature_mutation_zone,type_local_zone,nombre_pieces_principales_zone,prix_m2_zone
0,02/01/2020,Vente,85000.0,13400.0,AUBAGNE,Appartement,28.0,1.0,3035.714286,Vente,Appartement,1.0,3035.714286
1,09/01/2020,Vente,106000.0,13400.0,AUBAGNE,Appartement,34.0,1.0,3117.647059,Vente,Appartement,1.0,3117.647059
2,06/01/2020,Vente,72200.0,13600.0,LA CIOTAT,Appartement,30.0,1.0,2406.666667,Vente,Appartement,1.0,2406.666667
3,09/01/2020,Vente,77020.0,13600.0,LA CIOTAT,Appartement,22.0,1.0,3500.909091,Vente,Appartement,1.0,3500.909091
4,03/01/2020,Vente,165000.0,13600.0,LA CIOTAT,Appartement,27.0,1.0,6111.111111,Vente,Appartement,1.0,6111.111111


In [12]:
df_student_final.to_csv("Output/df_student_final.csv", index=False)

In [ ]:
prix_moyen_ville_final = (
    df_student_final.groupby('commune')['prix_m2_typique']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

prix_moyen_ville_final.head(15)


,commune,prix_m2_typique
0,LIAUSSON,17766.666667
1,LES PLANS,13666.666667
2,PARIS 06,13029.200413
3,PARIS 07,12729.928817
4,PARIS 04,12552.444022
5,LUZINAY,12292.609524
6,LIMAS,12068.965517
7,PARIS 05,12011.063998
8,LE POUJOL-SUR-ORB,11941.101449
9,PARIS 01,11716.552081
